<a href="https://colab.research.google.com/github/hongxuzhang/courses/blob/master/a3_Q2_colab_zhanghongxu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Q2

Adapted from Problem 3.15 of Tsay (2005)] Consider the percentage daily log returns of GM stock and the S&P
500 index from 2003 to 2021, but we shall investigate whether the volatility of GM stock has any contribution
in modeling the S&P 500 index volatility. Follow the steps below to perform the analysis.

(a) Fit a GARCH(1,1) model with generalized error distribution to the percentage log returns of GM stock.
Denote the fitted volatility by gmvol. Build an adequate GARCH model plus gmvol as the exogenous variable for the log S&P return series. Write down the fitted model.

(b) Is the volatility of GM stock returns helpful in modeling the volatility of the S&P index returns? Why?

In [2]:
# change directory from google colab into google drive, connect to google drive first
from google.colab import drive
drive.mount('/content/drive')

# change the present working directory
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
#import warnings

plt.style.use('seaborn')
# plt.style.use('seaborn-colorblind') #alternative
plt.rcParams['figure.figsize'] = [8, 4.5]
plt.rcParams['figure.dpi'] = 300
#warnings.simplefilter(action='ignore', category=FutureWarning)

Mounted at /content/drive


<ipython-input-2-0d8c9de16522>:15: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [4]:
!pip install arch;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.4/983.4 kB 5.4 MB/s eta 0:00:00


In [13]:
import pandas as pd
import yfinance as yf
from arch import arch_model
import numpy as np

In [8]:
#### for GM
RISKY_ASSET = 'GM'
START_DATE = '2003-01-01'
END_DATE = '2021-12-31'

gm = yf.download(RISKY_ASSET,
                 start=START_DATE,
                 end=END_DATE)

print(f'Downloaded {gm.shape[0]} rows of data.')

# Calculate daily returns (mutiply 100 to avoid convergence warnings due to very small numbers)
gm_returns = 100 * gm['Adj Close'].pct_change().dropna()

[*********************100%%**********************]  1 of 1 completed

Downloaded 2798 rows of data.


In [27]:
# # rename return
# gm_returns.name = 'asset_returns'
# print(f'Average return: {round(gm_returns.mean(), 2)}%')

# # plot the daily returns and add the title
# gm_returns.plot(title=f'{RISKY_ASSET} gm_returns: {START_DATE} - {END_DATE}');

# plt.tight_layout()
# plt.show()

In [21]:
### for S&P500
RISKY_ASSET = '^GSPC'
START_DATE = '2003-01-01'
END_DATE = '2021-12-31'

sp = yf.download(RISKY_ASSET,
                 start=START_DATE,
                 end=END_DATE)

print(f'Downloaded {sp.shape[0]} rows of data.')

sp_returns = 100 * sp['Adj Close'].pct_change()

sp_returns_log = np.log(sp_returns)
sp_returns_log = sp_returns_log.replace([np.inf, -np.inf], np.nan)

[*********************100%%**********************]  1 of 1 completed

Downloaded 4783 rows of data.



/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [28]:
# # # rename return
# sp_returns.name = 'asset_returns'
# print(f'Average return: {round(sp_returns.mean(), 2)}%')

# # plot the daily returns and add the title
# sp_returns.plot(title=f'{RISKY_ASSET} sp_returns: {START_DATE} - {END_DATE}');

# plt.tight_layout()
# plt.show()

In [24]:
### build GARCH model for GM
# define the model, we selected the zero-mean approach for comparision's sake
gm_model = arch_model(gm_returns_log.dropna(), mean='Zero', vol='GARCH', # specify the type of volatility model and set q=1
                   p=1, o=0, q=1)

In [26]:
garch_results = gm_model.fit()

Iteration:      1,   Func. Count:      5,   Neg. LLF: 1296259.6358286492
Iteration:      2,   Func. Count:     12,   Neg. LLF: 5343.729946083712
Iteration:      3,   Func. Count:     21,   Neg. LLF: 4883.438200921873
Iteration:      4,   Func. Count:     26,   Neg. LLF: 5464.5565171760745
Iteration:      5,   Func. Count:     31,   Neg. LLF: 4878.019993408226
Iteration:      6,   Func. Count:     36,   Neg. LLF: 4863.051684049252
Iteration:      7,   Func. Count:     41,   Neg. LLF: 4862.2820059697215
Iteration:      8,   Func. Count:     45,   Neg. LLF: 4862.276139736509
Iteration:      9,   Func. Count:     49,   Neg. LLF: 4862.276079248016
Iteration:     10,   Func. Count:     53,   Neg. LLF: 4862.2760760457395
Iteration:     11,   Func. Count:     57,   Neg. LLF: 4862.276072387276
Iteration:     12,   Func. Count:     60,   Neg. LLF: 4862.2760723868305
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4862.276072387276
            Iterations:

In [31]:
# Step 3: Obtain Fitted Volatility (gmvol)
gmvol = garch_results.conditional_volatility

# Step 4: Build GARCH Model for S&P 500 Index Returns
sp500_model = arch_model(sp_returns_log.dropna(), vol='Garch', p=1, q=1, x=gmvol)
sp500_results = sp500_model.fit()

# Step 5: Assess the Contribution of GM Stock Volatility
# Examine the estimated coefficients, including the coefficient for gmvol
print(sp500_results.summary())


Iteration:      1,   Func. Count:      6,   Neg. LLF: 10867.718241631144
Iteration:      2,   Func. Count:     15,   Neg. LLF: 34489961909.911156
Iteration:      3,   Func. Count:     24,   Neg. LLF: 640826.93695432
Iteration:      4,   Func. Count:     32,   Neg. LLF: 4360.842563806986
Iteration:      5,   Func. Count:     38,   Neg. LLF: 4359.630677359222
Iteration:      6,   Func. Count:     44,   Neg. LLF: 4356.253803185551
Iteration:      7,   Func. Count:     50,   Neg. LLF: 4356.040900105772
Iteration:      8,   Func. Count:     56,   Neg. LLF: 4347.060799389719
Iteration:      9,   Func. Count:     62,   Neg. LLF: 4346.51627803806
Iteration:     10,   Func. Count:     68,   Neg. LLF: 4346.501805350744
Iteration:     11,   Func. Count:     74,   Neg. LLF: 4346.500853472531
Iteration:     12,   Func. Count:     79,   Neg. LLF: 4346.500817253525
Iteration:     13,   Func. Count:     84,   Neg. LLF: 4346.500813151051
Iteration:     14,   Func. Count:     89,   Neg. LLF: 4346.500812

Summary:

It appears that the GARCH model has captured the mean level of the S&P 500 index returns well, as indicated by the significant coefficient for the mean term.

However, the volatility model's coefficients (omega, alpha[1]) are not statistically significant. This suggests that the volatility of GM stock returns, represented by gmvol, may not have a significant impact on modeling the volatility of the S&P 500 index returns, based on this particular GARCH model.

the beta[1] is significant, which suggests that the lagged squared residuals play an important role in explaining the persistence of volatility in the S&P 500 index returns, indicating a long-term memory effect in the volatility dynamics